In [270]:
########################## SET UP ##########################

response = 'rvl_ind_drp_prc4'

tuning_ind = 0

In [271]:
[k for k in initial_variables if 'rvl_ind_drp' in k]
# #purchase_attrition_ind
# #revolve_attrition_ind

['rvl_ind_drp_dol2',
 'rvl_ind_drp_prc2',
 'rvl_ind_drp_dol3',
 'rvl_ind_drp_prc3',
 'rvl_ind_drp_dol4',
 'rvl_ind_drp_prc4',
 'rvl_ind_drp_dol5',
 'rvl_ind_drp_prc5',
 'rvl_ind_drp_dol6',
 'rvl_ind_drp_prc6',
 'rvl_ind_drp_dol7',
 'rvl_ind_drp_prc7']

In [272]:
#%pylab inline
import dataiku
import dataiku.spark as dkuspark
import pyspark
from pyspark.sql import SQLContext

import pandas as pd
import seaborn as sns
import numpy as np
import re as re
import math
import pickle
import time
import sys
import datetime
#print(sys.version)

from matplotlib import pyplot as plt
from math import log
from datetime import datetime, timedelta
from numpy import sort

import scipy.stats as st
import scipy.special as spec

from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit,StratifiedKFold, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, precision_score, recall_score,confusion_matrix, roc_curve
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

from xgboost import XGBClassifier, plot_importance

import lightgbm as lgb
from lightgbm import LGBMClassifier

import warnings
warnings.filterwarnings("ignore")

In [273]:
# ################################ Loading Main Data Set ################################

# start = time.time()
# model_data_dk = dataiku.Dataset("model_data2")
# end = time.time()
# print("Dataiku execusion: ", int((end - start)/60), "minutes")

In [274]:
# start = time.time()
# #get_dataframe(sampling = 'random', ratio = 0.1)
# model_data = model_data_dk.get_dataframe()
# end = time.time()
# print("GetDataFrame execusion: ", int((end - start)/60), "minutes")

In [275]:
# ############################### Loading Cycle Data Set ################################

# start = time.time()
# model_cycle_data = dataiku.Dataset("model_cycle")
# model_cycle = model_cycle_data.get_dataframe()
# end = time.time()
# print("Execution Time for Cycle Data Was: ", (end - start)/60, "minutes")

# model_cycle.head()

In [276]:
# ############################### Loading Targets Data Set ################################

# start = time.time()
# multiple_targets_data = dataiku.Dataset("multiple_targets2")
# multiple_targets = multiple_targets_data.get_dataframe()
# end = time.time()
# print("Execution Time for Targets Data Was: ", (end - start)/60, "minutes")

In [277]:
# multiple_targets

In [278]:
# ################################ Loading Payments Data Set ################################

# start = time.time()
# model_payments_data = dataiku.Dataset("model_payments_data")
# model_payments = model_payments_data.get_dataframe()
# end = time.time()
# print("Execution Time for Payments Data Was: ", (end - start)/60, "minutes")

In [279]:
# model_payments.head()

In [280]:
# ############################## Getting New Fields from Payments Data ##############################################

# ## Group FIs
# model_payments.loc[model_payments['fi_nm'].str.contains('ROYAL|AVION'), 'fi_group' ] = 'rbc'
# model_payments.loc[model_payments['fi_nm'].str.contains('CANADIAN TIRE'), 'fi_group' ] = 'cad_tire'
# model_payments.loc[model_payments['fi_nm'].str.contains('BANK OF MONTREAL'), 'fi_group' ] = 'bmo'
# model_payments.loc[model_payments['fi_nm'].str.contains('CIBC'), 'fi_group' ] = 'cibc'
# model_payments.loc[model_payments['fi_nm'].str.contains('TD'), 'fi_group' ] = 'td'
# model_payments.loc[model_payments['fi_nm'].str.contains('MBNA'), 'fi_group' ] = 'mbna'
# model_payments.loc[model_payments['fi_nm'].str.contains('SCOTIA'), 'fi_group' ] = 'scotia'
# model_payments.loc[model_payments['fi_nm'].str.contains('AMERICAN EXPRESS'), 'fi_group' ] = 'amex'

# model_payments.loc[model_payments['fi_group'].isnull(), 'fi_group' ] = 'the_rest'

In [281]:
# def months_subtract(input_date,months_to_subtract):
    
#     input_year = int(input_date/100)
#     input_month = input_date%100 
    
#     if (input_month - months_to_subtract) > 0:
#         output_month = input_month - months_to_subtract
#         output_year = input_year
#     else: 
#         output_month = (input_month - months_to_subtract) + 12
#         output_year = input_year - 1
    
#     output_date = output_year*100 + output_month
        
#     return output_date

In [282]:
# now = 201804

# last_month = months_subtract(now,1)
# three_months_ago = months_subtract(now,3)
# six_months_ago =  months_subtract(now,6)
# twelve_months_ago =  months_subtract(now,12)

In [283]:
# fi_balances = pd.DataFrame()

# metrics = ['bal_amt', 'cl_amt', 'last_2_mth_spd_amt', 'rvl_bal' ] # and do totals for other metrics, maybe
# fi_groups = model_payments.fi_group.unique().tolist() 

# time_frames = [last_month, three_months_ago, six_months_ago, twelve_months_ago ]
# time_frame_names = ['1_month', '3_month_avg', '6_month_avg', '12_month_avg']


# for (time_frame, time_frame_name) in zip(time_frames, time_frame_names):
    
#     #for a certain time period..
#     certain_months = pd.DataFrame()
#     certain_months = model_payments[(model_payments['ym_id'] >= time_frame) & (model_payments['ym_id'] <= last_month )]
    
#     #pull all metrics for all FIs..
#     for metric in metrics:
        
#         #save them in "balances" data frame..
#         balances = pd.DataFrame()
#         balances = certain_months[['equifax_id', 'fi_group', metric]
#                                  ].groupby(['equifax_id', 'fi_group']).mean().unstack('fi_group').fillna(0).reset_index()

#         names_list = [metric + '_' + x + '_' + time_frame_name for x in fi_groups]
#         balances.columns = ['equifax_id'] + names_list

#         #and then join it back to the big dataframe "fi_balances" that will store all these metrics for all the loops
#         if fi_balances.shape[0] == 0:
#             fi_balances = balances
#         else:
#             fi_balances = fi_balances.merge(balances, on = ['equifax_id'], how = 'outer') 

In [284]:
# list(fi_balances) #rvl_bal_bmo_3month_avg

In [285]:
# model_data_fi = model_data.merge(fi_balances, on = ['equifax_id'], how  = 'left')

In [286]:
# #how many missing ocifs
# model_data_fi['ocif_id'].isnull().sum()

In [287]:
# model_data_exclusions = model_data_fi.loc[(model_data_fi['ocif_id'].isnull() == False)
#                                            & (model_data_fi['rvl_now_ind'] == 1)
#                                            & (model_data_fi['num_total_mnth'] == 12)
#                                            & (model_data_fi['num_pos_rvl_mnth']>5), :]

In [288]:
# print ('After the exclusion criteria, we reduced size of the data from', model_data.shape[0] ,
#        "rows to", model_data_exclusions.shape[0], 'rows')

In [289]:
# data = model_data_exclusions

In [290]:
# data

In [291]:
################################# EXPORTED DATA AS VC_ATTRITION_MODEL TABLE ####################################################

In [292]:
#importing the whole dataset for modelling

start = time.time()
data_dk = dataiku.Dataset("vc_attrition_model")
data = data_dk.get_dataframe()
end = time.time()
print("Execution Time: ", int((end - start)/60), "minutes")

Execution Time:  1 minutes


In [293]:
#importing dataset with different targets

start = time.time()
targets_dk = dataiku.Dataset("vc_multiple_targets")
targets = targets_dk.get_dataframe()
end = time.time()
print("Execution Time: ", int((end - start)/60), "minutes")

Execution Time:  0 minutes


In [294]:
#combine data with newly added targets
data = data.merge(targets, on = ['mba_acct_nbr'], how = 'left') 

In [295]:
data.shape

(736001, 450)

In [296]:
initial_variables = list(data)
variables_include_future = ['revolve_attrition_ind', 'purchase_attrition_ind'] + [k for k in initial_variables if 'rvl_ind_drp' in k]
variables_include_future.remove(response)

In [297]:
#data[['mba_acct_nbr','revolve_attrition_ind']].groupby('revolve_attrition_ind', as_index = False).count()

In [298]:
#list(data)

In [299]:
#create list of variables that have NULL values
all_variables = list(data)
nullval_variables = list()

for variable in all_variables:
    if data[variable].isnull().sum() > 0:
        nullval_variables.append(variable)

In [300]:
###################### Treat NULL with O ###########################

#create a list to treat variables to be 0 instead of NULLs
null_means_zero = ['bal', 'cnt', 'amt', 'nbr']
zero_for_null_variables = list()

for variable in nullval_variables:
    if any([x in variable for x in null_means_zero]):
        zero_for_null_variables.append(variable)
        
        
#some of them we don't want to treat with 0
to_remove = ['cntry_cd']
for item in to_remove:
    zero_for_null_variables.remove(item)
    

# treat with zero
data = data.copy()
for variable in zero_for_null_variables:
    data[variable].fillna(0, inplace = True)

In [301]:
#zero_for_null_variables

In [302]:
# rerun check for how many variables have NULL after treating many of them with 0
all_variables = list(data)
nullval_variables = list()

for variable in all_variables:
    if data[variable].isnull().sum() > 0:
        nullval_variables.append(variable)

In [303]:
#nullval_variables

In [304]:
#get variables that have nulls
null_counts = data.isnull().sum().reset_index()
variables_have_nulls = list(null_counts.loc[null_counts[0]>0,:]['index'] )

In [305]:
#and some other variables you want to delete
variables_not_needed = ['mth_indvdl_inc_amt', 'mth_hshld_inc_amt']
variable_ids =  ['ocif_id','cad_acct_id', 'mba_acct_nbr_cus']
variables_constants = ['num_total_mnth','rvl_now_ind' ]

In [306]:
# delete all of them
variables_to_delete = variables_have_nulls + variables_include_future + variables_not_needed + variable_ids + variables_constants

for variable in variables_to_delete:
    data = data.drop(variable,1) 

In [307]:
#variables_to_delete

In [308]:
#creating new variables on balances and transactions
balance_variables = ['cl_amt', 'crn_bal', 'olim_amt', 'pymt_cnt', 'pymt_amt', 'ccar_net_pur_amt', 'ccar_net_cash_amt',
                    'cyc_low_bal_amt','cyc_high_bal_amt', 'bal_rvl_amt' ]

transaction_variables = ['total_txn_amt', 'total_txn_cnt', 'fdbvr_txn_amt', 'grcry_txn_amt', 'gas_txn_amt', 
                         'clth_txn_amt', 'trvl_txn_amt' ]

first_mnth = ['3','6']
second_mnth = ['6', '12']

for variable in balance_variables + transaction_variables:
    for (first,second) in zip(first_mnth,second_mnth):
        
        divident = variable + '_'+ first +'month_avg'
        divisor = variable + '_'+ second +'month_avg'
        new_variable = variable + '_'+ first + '_'+ second +'month_avg'

        data.loc[data[divisor] !=0 ,new_variable]= data [divident]/data[divisor]
        data.loc[data[divisor] == 0 ,new_variable] = 0

In [309]:
# create new variables on payments data
variables = ['rvl_bal','last_2_mth_spd_amt', 'cl_amt', 'bal_amt']
fis = ['rbc','cad_tire', 'bmo', 'cibc', 'td', 'mbna', 'scotia', 'amex', 'the_rest'] 

first_mnth = ['3','6']
second_mnth = ['6', '12']

for variable in variables:
    for (first, second) in zip(first_mnth,second_mnth):
        for fi in fis:
            
            divident = variable + '_'+ fi + '_' + first + '_' +'month_avg'
            divisor = variable + '_'+ fi + '_' + second + '_' +'month_avg'
            new_variable = variable + '_'+ fi + '_' + first + '_'+ second +'month_avg'

            data.loc[data[divisor] !=0 ,new_variable]= data [divident]/data[divisor]
            data.loc[data[divisor] == 0 ,new_variable] = 0

In [310]:
# #rvl_bal_bmo_6_12month_avg
# variables = list(data)
# bmo_variables = list()
# for variable in variables:
#     if ('the_rest' in variable):
#         bmo_variables.append(variable)

In [311]:
#sort(bmo_variables)

In [312]:
# create new 
metrics = ['bal_amt', 'cl_amt', 'last_2_mth_spd_amt', 'rvl_bal']
fis = ['rbc','cad_tire', 'cibc', 'td', 'mbna', 'scotia', 'amex', 'the_rest']  #, 'bmo'
timelines = ['1_month','3_month_avg', '6_month_avg', '12_month_avg', '3_6month_avg', '6_12month_avg']

for metric in metrics:
    for timeline in timelines:
        
        all_field_name = metric + '_all_' + timeline
        data[all_field_name] = 0
        
        for fi in fis:
            
            fi_field_name = metric + '_' + fi + '_' + timeline
            data[all_field_name] += data[fi_field_name]

In [313]:
data.head()

,mba_acct_nbr,cpc_cd,airmiles_ind,cpc_type,reward_type,account_tenure_mnth,acct_state_cd,algnd_triad_scr,cl_amt,orig_cl_amt,cr_scr,cbr_scr,cash_lim_amt,co_ind,curr_cd,high_bal,tnr_mth_nbr,tib_cd,init_pyrl_acct_ind,pac_cust_ind,pad_cust_ind,rsp_cust_ind,resp_cust_ind,tfsa_cust_ind,init_pyrl_amt,adj_mtg_tot_bal,adj_loc_tot_bal,adj_pers_loan_tot_bal,cr_crd_tot_bal,lndg_tot_bal,adj_rgst_svg_tot_bal,adj_nrgst_svg_tot_bal,adj_edb_tot_bal,tot_rtl_bal,adj_mtg_acct_cnt,adj_loc_acct_cnt,adj_pers_loan_acct_cnt,cr_crd_acct_cnt,lndg_acct_cnt,adj_edb_acct_cnt,dpst_and_ivsm_cnt,prod_cnt,acct_cnt,nsf_last_6_mth_cnt,olb_rgst_ind,cr_crd_ann_rtl_amt,dcsd_ind,self_emp_ind,new_cdn_ind,cr_crd_prim_acct_cnt,cr_crd_prpaid_ind,cr_crd_fx_txn_ind,cr_crd_gas_spd_amt,cr_crd_tvl_spd_amt,cr_crd_entmt_spd_amt,cr_crd_grocery_spd_amt,stu_ind,prim_bnkg_cust_ind,appl_decl_ind,ytd_rrsp_ctb_amt,am_ind,forgn_addr_ind,bkrp_ind,jnt_non_edb_ind,preauth_pymt_txn_cnt,preauth_pymt_txn_amt,preauth_cr_crd_pymt_usg_ind,tot_12m_opic_fee_amt,cl_amt_1month,crn_bal_1month,olim_amt_1month,pdu_amt_1month,pymt_cnt_1month,pymt_amt_1month,ccar_net_pur_amt_1month,ccar_net_cash_amt_1month,ccar_net_bal_tfr_amt_1month,ccar_net_chq_amt_1month,prev_stmt_bal_1month,cyc_low_bal_amt_1month,cyc_high_bal_amt_1month,bal_rvl_amt_1month,cl_amt_3month,crn_bal_3month,olim_amt_3month,pymt_cnt_3month,pymt_amt_3month,ccar_net_pur_amt_3month,ccar_net_cash_amt_3month,cyc_low_bal_amt_3month,cyc_high_bal_amt_3month,bal_rvl_amt_3month,cl_amt_6month,crn_bal_6month,olim_amt_6month,pymt_cnt_6month,pymt_amt_6month,ccar_net_pur_amt_6month,ccar_net_cash_amt_6month,cyc_low_bal_amt_6month,cyc_high_bal_amt_6month,bal_rvl_amt_6month,cl_amt_9month,crn_bal_9month,olim_amt_9month,...,fdbvr_txn_amt_6_12month_avg,grcry_txn_amt_3_6month_avg,grcry_txn_amt_6_12month_avg,gas_txn_amt_3_6month_avg,gas_txn_amt_6_12month_avg,clth_txn_amt_3_6month_avg,clth_txn_amt_6_12month_avg,trvl_txn_amt_3_6month_avg,trvl_txn_amt_6_12month_avg,rvl_bal_rbc_3_6month_avg,rvl_bal_cad_tire_3_6month_avg,rvl_bal_bmo_3_6month_avg,rvl_bal_cibc_3_6month_avg,rvl_bal_td_3_6month_avg,rvl_bal_mbna_3_6month_avg,rvl_bal_scotia_3_6month_avg,rvl_bal_amex_3_6month_avg,rvl_bal_the_rest_3_6month_avg,rvl_bal_rbc_6_12month_avg,rvl_bal_cad_tire_6_12month_avg,rvl_bal_bmo_6_12month_avg,rvl_bal_cibc_6_12month_avg,rvl_bal_td_6_12month_avg,rvl_bal_mbna_6_12month_avg,rvl_bal_scotia_6_12month_avg,rvl_bal_amex_6_12month_avg,rvl_bal_the_rest_6_12month_avg,last_2_mth_spd_amt_rbc_3_6month_avg,last_2_mth_spd_amt_cad_tire_3_6month_avg,last_2_mth_spd_amt_bmo_3_6month_avg,last_2_mth_spd_amt_cibc_3_6month_avg,last_2_mth_spd_amt_td_3_6month_avg,last_2_mth_spd_amt_mbna_3_6month_avg,last_2_mth_spd_amt_scotia_3_6month_avg,last_2_mth_spd_amt_amex_3_6month_avg,last_2_mth_spd_amt_the_rest_3_6month_avg,last_2_mth_spd_amt_rbc_6_12month_avg,last_2_mth_spd_amt_cad_tire_6_12month_avg,last_2_mth_spd_amt_bmo_6_12month_avg,last_2_mth_spd_amt_cibc_6_12month_avg,last_2_mth_spd_amt_td_6_12month_avg,last_2_mth_spd_amt_mbna_6_12month_avg,last_2_mth_spd_amt_scotia_6_12month_avg,last_2_mth_spd_amt_amex_6_12month_avg,last_2_mth_spd_amt_the_rest_6_12month_avg,cl_amt_rbc_3_6month_avg,cl_amt_cad_tire_3_6month_avg,cl_amt_bmo_3_6month_avg,cl_amt_cibc_3_6month_avg,cl_amt_td_3_6month_avg,cl_amt_mbna_3_6month_avg,cl_amt_scotia_3_6month_avg,cl_amt_amex_3_6month_avg,cl_amt_the_rest_3_6month_avg,cl_amt_rbc_6_12month_avg,cl_amt_cad_tire_6_12month_avg,cl_amt_bmo_6_12month_avg,cl_amt_cibc_6_12month_avg,cl_amt_td_6_12month_avg,cl_amt_mbna_6_12month_avg,cl_amt_scotia_6_12month_avg,cl_amt_amex_6_12month_avg,cl_amt_the_rest_6_12month_avg,bal_amt_rbc_3_6month_avg,bal_amt_cad_tire_3_6month_avg,bal_amt_bmo_3_6month_avg,bal_amt_cibc_3_6month_avg,bal_amt_td_3_6month_avg,bal_amt_mbna_3_6month_avg,bal_amt_scotia_3_6month_avg,bal_amt_amex_3_6month_avg,bal_amt_the_rest_3_6month_avg,bal_amt_rbc_6_12month_avg,bal_amt_cad_tire_6_12month_avg,bal_amt_bmo_6_12month_avg,bal_amt_cibc_6_12month_avg,bal_amt_td_6_12month_avg,bal_amt_mbna_6_12

In [314]:
# change all indicators to be categorical variables except the response
all_variables = list(data)
all_variables.remove(response)

to_type = 'object'
for variable in all_variables:
    if 'ind' in variable:
        data.loc[:,variable] = data[variable].astype(to_type)

In [315]:
data.set_index('mba_acct_nbr', inplace=True)
data.head()

,cpc_cd,airmiles_ind,cpc_type,reward_type,account_tenure_mnth,acct_state_cd,algnd_triad_scr,cl_amt,orig_cl_amt,cr_scr,cbr_scr,cash_lim_amt,co_ind,curr_cd,high_bal,tnr_mth_nbr,tib_cd,init_pyrl_acct_ind,pac_cust_ind,pad_cust_ind,rsp_cust_ind,resp_cust_ind,tfsa_cust_ind,init_pyrl_amt,adj_mtg_tot_bal,adj_loc_tot_bal,adj_pers_loan_tot_bal,cr_crd_tot_bal,lndg_tot_bal,adj_rgst_svg_tot_bal,adj_nrgst_svg_tot_bal,adj_edb_tot_bal,tot_rtl_bal,adj_mtg_acct_cnt,adj_loc_acct_cnt,adj_pers_loan_acct_cnt,cr_crd_acct_cnt,lndg_acct_cnt,adj_edb_acct_cnt,dpst_and_ivsm_cnt,prod_cnt,acct_cnt,nsf_last_6_mth_cnt,olb_rgst_ind,cr_crd_ann_rtl_amt,dcsd_ind,self_emp_ind,new_cdn_ind,cr_crd_prim_acct_cnt,cr_crd_prpaid_ind,cr_crd_fx_txn_ind,cr_crd_gas_spd_amt,cr_crd_tvl_spd_amt,cr_crd_entmt_spd_amt,cr_crd_grocery_spd_amt,stu_ind,prim_bnkg_cust_ind,appl_decl_ind,ytd_rrsp_ctb_amt,am_ind,forgn_addr_ind,bkrp_ind,jnt_non_edb_ind,preauth_pymt_txn_cnt,preauth_pymt_txn_amt,preauth_cr_crd_pymt_usg_ind,tot_12m_opic_fee_amt,cl_amt_1month,crn_bal_1month,olim_amt_1month,pdu_amt_1month,pymt_cnt_1month,pymt_amt_1month,ccar_net_pur_amt_1month,ccar_net_cash_amt_1month,ccar_net_bal_tfr_amt_1month,ccar_net_chq_amt_1month,prev_stmt_bal_1month,cyc_low_bal_amt_1month,cyc_high_bal_amt_1month,bal_rvl_amt_1month,cl_amt_3month,crn_bal_3month,olim_amt_3month,pymt_cnt_3month,pymt_amt_3month,ccar_net_pur_amt_3month,ccar_net_cash_amt_3month,cyc_low_bal_amt_3month,cyc_high_bal_amt_3month,bal_rvl_amt_3month,cl_amt_6month,crn_bal_6month,olim_amt_6month,pymt_cnt_6month,pymt_amt_6month,ccar_net_pur_amt_6month,ccar_net_cash_amt_6month,cyc_low_bal_amt_6month,cyc_high_bal_amt_6month,bal_rvl_amt_6month,cl_amt_9month,crn_bal_9month,olim_amt_9month,pymt_cnt_9month,...,fdbvr_txn_amt_6_12month_avg,grcry_txn_amt_3_6month_avg,grcry_txn_amt_6_12month_avg,gas_txn_amt_3_6month_avg,gas_txn_amt_6_12month_avg,clth_txn_amt_3_6month_avg,clth_txn_amt_6_12month_avg,trvl_txn_amt_3_6month_avg,trvl_txn_amt_6_12month_avg,rvl_bal_rbc_3_6month_avg,rvl_bal_cad_tire_3_6month_avg,rvl_bal_bmo_3_6month_avg,rvl_bal_cibc_3_6month_avg,rvl_bal_td_3_6month_avg,rvl_bal_mbna_3_6month_avg,rvl_bal_scotia_3_6month_avg,rvl_bal_amex_3_6month_avg,rvl_bal_the_rest_3_6month_avg,rvl_bal_rbc_6_12month_avg,rvl_bal_cad_tire_6_12month_avg,rvl_bal_bmo_6_12month_avg,rvl_bal_cibc_6_12month_avg,rvl_bal_td_6_12month_avg,rvl_bal_mbna_6_12month_avg,rvl_bal_scotia_6_12month_avg,rvl_bal_amex_6_12month_avg,rvl_bal_the_rest_6_12month_avg,last_2_mth_spd_amt_rbc_3_6month_avg,last_2_mth_spd_amt_cad_tire_3_6month_avg,last_2_mth_spd_amt_bmo_3_6month_avg,last_2_mth_spd_amt_cibc_3_6month_avg,last_2_mth_spd_amt_td_3_6month_avg,last_2_mth_spd_amt_mbna_3_6month_avg,last_2_mth_spd_amt_scotia_3_6month_avg,last_2_mth_spd_amt_amex_3_6month_avg,last_2_mth_spd_amt_the_rest_3_6month_avg,last_2_mth_spd_amt_rbc_6_12month_avg,last_2_mth_spd_amt_cad_tire_6_12month_avg,last_2_mth_spd_amt_bmo_6_12month_avg,last_2_mth_spd_amt_cibc_6_12month_avg,last_2_mth_spd_amt_td_6_12month_avg,last_2_mth_spd_amt_mbna_6_12month_avg,last_2_mth_spd_amt_scotia_6_12month_avg,last_2_mth_spd_amt_amex_6_12month_avg,last_2_mth_spd_amt_the_rest_6_12month_avg,cl_amt_rbc_3_6month_avg,cl_amt_cad_tire_3_6month_avg,cl_amt_bmo_3_6month_avg,cl_amt_cibc_3_6month_avg,cl_amt_td_3_6month_avg,cl_amt_mbna_3_6month_avg,cl_amt_scotia_3_6month_avg,cl_amt_amex_3_6month_avg,cl_amt_the_rest_3_6month_avg,cl_amt_rbc_6_12month_avg,cl_amt_cad_tire_6_12month_avg,cl_amt_bmo_6_12month_avg,cl_amt_cibc_6_12month_avg,cl_amt_td_6_12month_avg,cl_amt_mbna_6_12month_avg,cl_amt_scotia_6_12month_avg,cl_amt_amex_6_12month_avg,cl_amt_the_rest_6_12month_avg,bal_amt_rbc_3_6month_avg,bal_amt_cad_tire_3_6month_avg,bal_amt_bmo_3_6month_avg,bal_amt_cibc_3_6month_avg,bal_amt_td_3_6month_avg,bal_amt_mbna_3_6month_avg,bal_amt_scotia_3_6month_avg,bal_amt_amex_3_6month_avg,bal_amt_the_rest_3_6month_avg,bal_amt_rbc_6_12month_avg,bal_amt_cad_tire_6_12month_avg,bal_amt_bmo_6_12month_avg,bal_amt_cibc_6_12month_avg,bal_amt_td_6_12month_avg,bal_amt_mbna_6

In [316]:
# ############# logging variables ##################
# non_cat_var = data.select_dtypes(include = ['int64', 'float64']).columns.tolist()
# non_cat_var.remove(response)

# for variable in non_cat_var:
#     min_value =  data[variable].min() 
#     data[variable] = data[variable].apply(lambda x: log(x+1-min_value))

In [317]:
y = data.loc[:,response]
X = data.drop (response, 1)

In [318]:
random = 768
test_prc = 0.3

X_train , X_test, y_train, y_test = train_test_split(X, y, test_size = test_prc, random_state = random)

In [319]:
data_train = X_train
data_train[response] = y_train

data_test = X_test
data_test[response] = y_test

In [320]:
def create_dummies(data, to_print):
    
    categorical_variables = data.select_dtypes(include = ['category', 'object']).columns.tolist()
    dummy_variables = pd.get_dummies(data.loc[:,categorical_variables], drop_first = True)

    data = pd.merge(data, dummy_variables, left_index = True, right_index = True)
    data_out = data.drop(categorical_variables, axis = 1)
    
    if to_print == 1:
    
        print ("List of all categorical variables: ", "\n")
        print (*categorical_variables, sep = "\n")
        print (" ")

        print ("After creating dummy variables, the list looks like this: ", "\n")
        print ( *data_out.columns.tolist(), sep = "\n")
        print (" ")
        
    return data_out

In [321]:
data_train = create_dummies(data_train,0)
data_test = create_dummies(data_test,0)

In [322]:
all_variables = list (data_train)
all_variables.remove(response)

supes_important = ['rvl_bal_the_rest_1_month', 'cr_crd_tot_bal', 'total_txn_amt_12month_avg']
#important_from_model = pd.Series(best_classifier.feature_importances_, index=X_test.columns).nlargest(5).reset_index()['index'].tolist()

In [323]:
######################### AB Featue Selection ########################################


y_train = data_train.loc[:, response]
X_train = data_train.loc[:, all_variables]

treeCL = DecisionTreeClassifier(criterion="entropy")
treeCL = treeCL.fit(X_train,y_train)

model = SelectFromModel(treeCL,prefit=True)

mask = model.get_support()
important_variables = X_train.columns[mask]


######################### VC Featue Selection ########################################

# X_train_fs, X_test_fs, y_train_fs, y_test_fs = train_test_split(X_train, y_train, test_size=0.30, random_state = 1209)

# model = ExtraTreesClassifier() 
# model.fit(X_train_fs, y_train_fs)


# # make predictions for test data and evaluate
# y_pred_fs = model.predict(X_test_fs)

# accuracy = accuracy_score(y_test_fs, y_pred_fs)
# print("Accuracy:", (accuracy * 100.0))

# # Fit model using each importance as a threshold
# #thresholds = sort(np.random.uniform(1e-4,1e-2,10)) #sort(model.feature_importances_)
# thresholds = []

# for thresh in thresholds:
    
#     # select features using threshold
#     selection = SelectFromModel(model, threshold=thresh, prefit=True)
#     select_X_train_fs = selection.transform(X_train_fs)
    
#     # train model
#     selection_model = ExtraTreesClassifier() 
#     selection_model.fit(select_X_train_fs, y_train_fs)
    
#     # eval model
#     select_X_test_fs = selection.transform(X_test_fs)
#     y_pred_fs = selection_model.predict(select_X_test_fs)

#     accuracy = accuracy_score(y_test_fs, y_pred_fs)
#     roc_auc = roc_auc_score(y_test_fs, y_pred_fs)
#     print("Thresh=%.3f, n=%d, ROC: %.2f%%" % (thresh, select_X_train_fs.shape[1], roc_auc*100.0))

In [324]:
#you can use all_variables, important_variables, or supes_important 

variables_to_include = important_variables

y_train = data_train.loc[:, response]
X_train = data_train.loc[:, variables_to_include]

y_test = data_test.loc[:, response]
X_test = data_test.loc[:, variables_to_include]

In [325]:
# ############################ TUNNING GRADIENT BOOSTING ################################################ 

# #https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/
# # consider using GridSearchCV

# gb = GradientBoostingClassifier()

# row_num = X_train.shape[0]

# learning_rate_start = 0.1 #0.03
# learning_rate_end = 0.4 #0.3

# n_estimators_start = 3
# n_estimators_end =  40 

# max_depth_start = 3
# max_depth_end = 40 

# # min_samples_split_start = round(0.01*row_num)
# # min_samples_split_end = round(0.03*row_num)

# # min_samples_leaf_start = 1
# # min_samples_leaf_end = 20 


# random_grid = {  
#                 "learning_rate": st.uniform (learning_rate_start, learning_rate_end),
#                 "n_estimators": st.randint (n_estimators_start, n_estimators_end),
#                 "max_depth": st.randint (max_depth_start, max_depth_end)
#             }


# # Random search of parameters, using 3 fold cross validation, 
# # search across 100 different combinations, and use all available cores
# gb_random = RandomizedSearchCV(estimator = gb, 
#                                            param_distributions = random_grid, 
#                                            n_iter = 10, cv = 3, scoring = 'f1', verbose=True, 
#                                            random_state=342, n_jobs =-1)


# start = time.time()
# gb_random.fit(X_train,y_train)
# end = time.time()

# print("Execution Time Was: ", end - start, "seconds")

# GradientBoostingClassifierTuned = GradientBoostingClassifier()
# GradientBoostingClassifierTuned.set_params(**gb_random.best_params_)
# GradientBoostingClassifierTuned.get_params()
# ############################################################################################

In [326]:
# GradientBoostingClassifierTuned = GradientBoostingClassifier(learning_rate = 0.2284706627804977,
#                                                              max_depth = 22,
#                                                              n_estimators= 20)

In [327]:
# ############### XGB Parameter Tunning ################

# #https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/
# # consider using GridSearchCV

# xgb = XGBClassifier()

# row_num = X_train.shape[0]

# learning_rate_start = 0.05
# learning_rate_end = 0.4

# n_estimators_start = 3
# n_estimators_end =  40

# max_depth_start = 3
# max_depth_end = 40

# one_to_left = st.beta(10, 1)  
# from_zero_positive = st.expon(0, 50)


# random_grid = {  
#     "n_estimators": st.randint(n_estimators_start, n_estimators_end),
#     "max_depth": st.randint(3, 100),
#     "learning_rate": st.uniform(learning_rate_start, learning_rate_end)
# #    "colsample_bytree": one_to_left,
# #    "subsample": one_to_left,
# #    "gamma": st.uniform(0, 10),
# #    'reg_alpha': from_zero_positive,
# #    "min_child_weight": from_zero_positive,
# }

# # random_grid = {  
# #     "n_estimators": st.randint(n_estimators_start, n_estimators_end),
# #     "max_depth": st.randint(max_depth_start, max_depth_end),
# #     "learning_rate": st.uniform(learning_rate_start, learning_rate_end),
# #     "colsample_bytree": one_to_left,
# #     "subsample": one_to_left,
# #     "gamma": st.uniform(0, 10),
# #     'reg_alpha': from_zero_positive,
# #     "min_child_weight": from_zero_positive
# # }

# # Random search of parameters, using 3 fold cross validation, 
# # search across 100 different combinations, and use all available cores
# xgb_random = RandomizedSearchCV(estimator = xgb, 
#                                            param_distributions = random_grid, 
#                                            n_iter = 50, cv = 10, scoring = 'f1', verbose=False, 
#                                            random_state=42)


# start = time.time()
# xgb_random.fit(X_train,y_train)
# end = time.time()

# print("Execution Time Was: ", end - start, "seconds")

# XGBClassifierTuned = XGBClassifier()
# XGBClassifierTuned.set_params(**xgb_random.best_params_)
# XGBClassifierTuned.get_params()
# #####################################################################################

In [328]:
# ############### Light GBM Parameter Tunning ################

# lgb = LGBMClassifier()

# #row_num = X_train.shape[0]

# random_grid = {  
#     "num_leaves": st.randint(3,  1000),
#     "max_depth": st.randint(3, 100),
#     "min_data_in_leaf": st.randint(50, 5000)
# }


# # Random search of parameters, using 3 fold cross validation, 
# # search across 100 different combinations, and use all available cores
# lgb_random = RandomizedSearchCV(estimator = lgb, 
#                                            param_distributions = random_grid, 
#                                            n_iter = 50, cv = 10, scoring = 'f1', verbose=False, 
#                                            random_state=42)


# start = time.time()
# lgb_random.fit(X_train,y_train)
# end = time.time()

# print("Execution Time Was: ", end - start, "seconds")

# LGBMClassifierTuned = LGBMClassifier()
# LGBMClassifierTuned.set_params(**lgb_random.best_params_)
# #LGBMClassifierTuned.get_params()
# #####################################################################################

In [329]:
# ############### Random Forest Parameter Tunning ################

# rf = RandomForestClassifier()
# #rf.get_params()

# row_num = X_train.shape[0]

# n_estimators_start = 5
# n_estimators_end =  150

# max_depth_start = 5
# max_depth_end = 40

# # min_samples_split_start =  round(0.01*row_num) #2
# # min_samples_split_end = round(0.03*row_num) #10

# min_samples_leaf_start = 2
# min_samples_leaf_end = 40


# max_features = ['auto', 'sqrt']
# bootstrap = [True, False]

# # Create the random grid
# random_grid = {'n_estimators': st.randint (n_estimators_start, n_estimators_end),
#                'max_depth': st.randint (max_depth_start, max_depth_end),
#                'min_samples_leaf': st.randint (min_samples_leaf_start, min_samples_leaf_end)
#               }


# rf_random = RandomizedSearchCV(estimator = rf, 
#                                            param_distributions = random_grid, 
#                                            n_iter = 50, cv = 5, scoring = 'recall', verbose=True, 
#                                            random_state=42)

# start = time.time()
# rf_random.fit(X_train,y_train)
# end = time.time()

# print("Execution Time Was: ", end - start, "seconds")

# RandomForestClassifierTuned = RandomForestClassifier()
# RandomForestClassifierTuned.set_params(**rf_random.best_params_)

In [330]:
#GradientBoostingClassifier  - 50 roc on training #####slow
#Random Forest: 91.84 roc on training #######fast

In [ ]:
best_classifier =  GradientBoostingClassifier() #RandomForestClassifier() # GradientBoostingClassifier() XGBClassifier() 

start = time.time()
best_classifier.fit(X_train, y_train)
end = time.time()

model_ran = int(end - start)

print("Execution Time Was: ", model_ran, "seconds")

y_pred_test = best_classifier.predict(X_test)
y_pred_train = best_classifier.predict(X_train)

y_pred_prob_test = best_classifier.predict_proba(X_test)
y_pred_prob_train = best_classifier.predict_proba(X_train)

In [ ]:
acc_test = round(100*accuracy_score(y_test, y_pred_test),2)
acc_train = round(100*accuracy_score(y_train, y_pred_train),2)

roc_test = round(100*roc_auc_score(y_test, y_pred_test),2)
roc_train = round(100*roc_auc_score(y_train, y_pred_train),2)

tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, y_pred_test).ravel()
tn_train, fp_train, fn_train, tp_train = confusion_matrix(y_train, y_pred_train).ravel()

recall_test = round(100*tp_test/(tp_test+fn_test),2)
precision_test = round(100*tp_test/(tp_test+fp_test),2)

recall_train = round(100*tp_train/(tp_train+fn_train),2)
precision_train = round(100*tp_train/(tp_train+fp_train),2)

print(
       " Accuracy on Testing:", acc_test, "%\n", 
       "ROC on Testing:", roc_test, "%\n",
      "Recall on Testing:",recall_test, "%\n",
      "Precision on Testing:", precision_test, "%\n",
      "##############################", "\n",
      "Accuracy on Training:", acc_train, "%\n",
      "ROC on Training: ", roc_train, "%\n",
      "Recall on Training:", recall_train, "%\n",
      "Precision on Training:", precision_train, "%")

In [ ]:
results = pd.DataFrame(columns = ['Run_date', 'Response', 'Method', 'Tuning', 'n_estimators', 'max_depth', 'min_samples_leaf',
                                  'learning_rate','Variables', 'Most_important_var',
                                  'Importance_of_it', 'Test_prc', 'Model_ran_sec',
                                 'Recall_train', 'Recall_test', 'Precision_train', 'Precision_test',
                                 'Accuracy_train', 'Accuracy_test', 'ROC_train', 'ROC_test'
                                 ])

In [ ]:
#get some metrics for the results table

method =  str(type(best_classifier))
method = method.split('.')[3].replace(">", "").replace("'", "")

import datetime
now = datetime.datetime.now()
now = now.strftime("%Y-%m-%d %H:%M")

most_important = pd.Series(best_classifier.feature_importances_,
                           index=X_test.columns).nlargest(1).reset_index()['index'].tolist()[0]

its_importance = pd.Series(best_classifier.feature_importances_,
                           index=X_test.columns).nlargest(1).reset_index()[0].tolist()[0]

parameters = str(best_classifier.get_params()).split(',')

n_estimators = [k for k in parameters if 'n_estimators' in k][0].split(':')[1].replace(' ','')
max_depth = [k for k in parameters if 'max_depth' in k][0].split(':')[1].replace(' ','')
min_samples_leaf = [k for k in parameters if 'min_samples_leaf' in k][0].split(':')[1].replace(' ','')
learning_rate = [k for k in parameters if 'learning_rate' in k][0].split(':')[1].replace(' ','')

#add a new record in the results table

results = results.append({'Run_date': now,
                'Response': response,
                'Method': method,
                'Tuning': tuning_ind,
                'n_estimators':n_estimators,
                'max_depth':max_depth,
                'min_samples_leaf':min_samples_leaf,
                'learning_rate':learning_rate,
                'Variables': variables_to_include.shape[0],
                'Most_important_var':most_important,
                'Importance_of_it': its_importance,
                'Test_prc':test_prc,
                'Model_ran_sec': model_ran,
                'Recall_train':recall_train,
                'Recall_test': recall_test,
                'Precision_train':precision_train,
                'Precision_test':precision_test,
                'Accuracy_train': acc_train,
                'Accuracy_test': acc_test,
                'ROC_train':roc_train,
                'ROC_test':roc_test
               }, ignore_index = True)


In [ ]:
results

In [ ]:
results_dk = dataiku.Dataset("results")
results = results_dk.get_dataframe()

In [ ]:
################### Display Decision Tree ###################
# from sklearn.tree import DecisionTreeClassifier, export_graphviz
# from sklearn import tree

# from IPython.display import SVG
# from graphviz import Source
# from IPython.display import display

#estimator = best_classifier.estimators_[5]
# graph = Source(tree.export_graphviz(estimator, out_file=None, filled = True))
# display(SVG(graph.pipe(format='svg')))

In [ ]:
#plot = plot_importance(best_classifier, max_num_features=15)
features_plot = pd.Series(best_classifier.feature_importances_, index=X_test.columns).nlargest(15).plot(kind='barh')

In [ ]:
# #ROC Curve: Test
# fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob_test[:,1])

# plt.clf()
# plt.plot(fpr, tpr)
# plt.xlabel('FPR')
# plt.ylabel('TPR')
# plt.title('ROC curve')
# plt.show()

In [ ]:
# #ROC Curve: Train
# fpr, tpr, thresholds = roc_curve(y_train, y_pred_train[:,1])

# plt.clf()
# plt.plot(fpr, tpr)
# plt.xlabel('FPR')
# plt.ylabel('TPR')
# plt.title('ROC curve')
# plt.show()

In [ ]:
# confusion_matrix(y_test, y_pred_test)

In [ ]:
# print(tn_test, fp_test, fn_test, tp_test )